In [3]:
# https://medium.com/@aminajavaid30/building-a-rag-system-synthesis-67f36efa7c35

# Data Ingestion & Retrieval
import bs4
import re
import os
import getpass
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain_community.document_loaders import GithubFileLoader
from bs4 import BeautifulSoup



In [4]:
# Load a github repo

if not os.environ.get("GITHUB_PAT"):
  os.environ["GITHUB_PAT"] = getpass.getpass("Enter Github PAT: ")

async def github_repo(repo_name: str) -> list[Document]:
    loader = GithubFileLoader(
        repo=repo_name, # the repo name
        access_token= os.environ["GITHUB_PAT"],
        github_api_url="https://api.github.com",
        branch ="master", 
        file_filter=lambda file_path: file_path.endswith(
            ".md"
        ),  # load all markdowns files.
    )
    github_docs = []
    async for doc in loader.alazy_load():
        github_docs.append(doc)
    return github_docs

In [5]:
# Load a website reccusivly and split it into chunks
def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()

async def website_parser(base_url: str) -> list[Document]:
    loader = RecursiveUrlLoader(
        base_url,
        extractor=bs4_extractor,
        max_depth=2,
        prevent_outside=True,
        # use_async=False,
        # extractor=None,
        # metadata_extractor=None,
        # exclude_dirs=(),
        # timeout=10,
        # check_response_status=True,
        # continue_on_failure=True,
        # 
        # base_url=None,
        # ...
    )
    site_docs = []
    async for doc in loader.alazy_load():
        site_docs.append(doc)
    return site_docs

In [6]:
# Initialize the OpenAI embedding model
import getpass
import os

from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

azureEmbeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://oai-bim42-test-fr-ai.openai.azure.com",
    azure_deployment="text-embedding-3-small",
    openai_api_version="2025-01-01-preview",
)

# Initialize the recursive character text splitter
text_splitter = RecursiveCharacterTextSplitter(              
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

# Initialize with an embedding model
vector_store = InMemoryVectorStore(embedding=azureEmbeddings)

In [7]:
# Define a list of site to parse
urls = ["https://bimdata.io/", "https://developers.bimdata.io/"]

# Iterate over each urls in the list
for url in urls:
    # Retrive the documents
    website_docs = await website_parser(url)
    website_docs
    print(f"{len(website_docs)} docs in {url}")
    # Split the documents into chunks
    chunks = text_splitter.split_documents(website_docs)
    print(f"{len(chunks)} chunks in {url}")
    # Add embeddings into the vector store
    vector_store.add_documents(documents=chunks)


C:\Users\smoreau\AppData\Local\Temp\ipykernel_22584\2916439140.py:3: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html, "lxml")
c:\Users\smoreau\AppData\Local\anaconda3\Lib\site-packages\langchain_community\document_loaders\recursive_url_loader.py:43: XMLParsedAsHTMLWarning: It looks like you're us

32 docs in https://bimdata.io/
893 chunks in https://bimdata.io/
6 docs in https://developers.bimdata.io/
26 chunks in https://developers.bimdata.io/


In [8]:
# Define a list of github repo to parse
repos = ["bimdata/platform", "bimdata/platform-back"]

# Iterate over each urls in the list
for repo in repos:
    # Retrive the documents
    github_docs = await github_repo(repo)
    print(f"{len(github_docs)} docs in {repo}")
    # Split the documents into chunks
    chunks = text_splitter.split_documents(github_docs)
    print(f"{len(chunks)} chunks in {repo}")
    # Add embeddings into the vector store
    vector_store.add_documents(documents=chunks)

3 docs in bimdata/platform
140 chunks in bimdata/platform
1 docs in bimdata/platform-back
3 chunks in bimdata/platform-back


In [9]:
# Retrieve relevant information using similarity search
retriever = vector_store.as_retriever() # uses similarity search by default

In [10]:
# Q/A Retrieval Chain

from langchain_openai import AzureChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize the Azure AI LLM with the specific model
llm = AzureChatOpenAI(
    azure_endpoint="https://oai-bim42-test-fr-ai.openai.azure.com",
    azure_deployment="gpt-4o-mini",
    openai_api_version="2025-01-01-preview",
)

# Define the system prompt that instructs the LLM how to answer questions based on retrieved context
system_prompt = (
    "You are a specialist of Building Information modeling looking for a web-based building model viewer"    
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question."
    "Add sources like links to your answer."
    "If you don't know the answer, say that you don't know."
    "Use four sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"  # Placeholder for the retrieved context
)

# Create a chat prompt template with a system message and human message
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),  # System message contains instructions and context
        ("human", "{input}"),  # Human message includes the user's input question
    ]
)

# Create a document chain that combines the LLM with the prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Combine the retrieval chain with the question-answering chain 
# The retrieval chain retrieves relevant documents and feeds them into the question-answering chain
retrieval_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
# Invoke the chain with a question, and the retriever will provide context for the LLM to generate an answer
response = retrieval_chain.invoke({"input": "Please describe the software editor behind this solution (name, description, history, "})
print(response['answer'])

I don't have information about the specific software editor behind the BIMData solution. If you are looking for details about a particular company or its history, I recommend checking their official website or relevant industry resources for comprehensive information.


In [12]:
response = retrieval_chain.invoke({"input": "What is the price or licencing model of the solution?"})
print(response['answer'])

The pricing for the BIMData solutions varies based on the chosen model. They offer a free Starter plan at €0 per year, a Professional plan starting at €540 per year, and an Enterprise plan starting at €2500 per year. The On-Premise option is available at customizable pricing based on client requirements. For more details, you can visit their website: [BIMData Pricing](https://bimdata.io).


In [13]:
response = retrieval_chain.invoke({"input": "Can you list all the input format accepted by the solution?"})
print(response['answer'])

The solution accepts various input formats, including number, password, range, search, tel, text, time, url, week, date, datetime, datetime-local, email, month, and select options as well as text areas. If you need more specific details or additional formats, please specify further.


In [14]:
response = retrieval_chain.invoke({"input": "What is the format used in the web viewer ?"})
print(response['answer'])

The web-based building model viewer typically uses the IFC (Industry Foundation Classes) format. This format is widely recognized in Building Information Modeling (BIM) for enabling interoperability between different software applications. For more specific details, you may need to refer to the viewer's documentation.


In [15]:
response = retrieval_chain.invoke({"input": "What is the WexBIM data format?"})
print(response['answer'])

The WexBIM data format is an open, lightweight data format used for building information modeling (BIM) that facilitates the exchange of 3D models and information related to them. It enables interoperability among different software tools and platforms within the BIM ecosystem. For more detailed information, you can refer to resources on BIMData and their tools related to open BIM: [BIMData](https://www.bimdata.io).


In [16]:
response = retrieval_chain.invoke({"input": "What are the main programming language used in the solution?"})
print(response['answer'])

The context provided does not specify the main programming languages used in the BIMData.io solution. Therefore, I don't know the answer to your question.


In [17]:
response = retrieval_chain.invoke({"input": "What are the licences for BIMData?"})
print(response['answer'])

KeyboardInterrupt: 